<a href="https://colab.research.google.com/github/Vinod1403/IIIT-Hyderabad-Tutorials/blob/main/1_STP_Project_Module_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2: Appreciating, Interpreting and Visualizing Data

**Project: Understanding Customer Segments for Targeted Marketing**

Introduction: The Power of Customer Segmentation
---



Welcome to your Module 2 project!

In today's competitive landscape, understanding your customers is paramount for any business. Generic marketing strategies often fall flat, but by truly appreciating the diverse needs and behaviors within your customer base, businesses can create more effective, personalized experiences. This process is known as **customer segmentation**,

Customer segmentation involves dividing a broad customer base into subgroups of consumers who have common needs, interests, and priorities. By segmenting customers, companies can:

* **Tailor Marketing Messages:** Design specific campaigns that resonate with each group.
* **Optimize Product Development:** Create products and services that meet the unique demands of different segments.
* **Improve Customer Service:** Provide support that addresses common issues for particular groups.
* **Identify High-Value Customers:** Focus resources on segments that drive the most revenue.
* **Predict Churn:** Identify customers at risk of leaving and intervene proactively.

In this project, your task is to analyze a dataset of customer activity, use dimensionality reduction techniques to visualize customer behavior, and ultimately identify distinct customer segments. This will demonstrate how data visualization can provide actionable insights for business strategy, even without deep domain expertise at the outset.

We will first focus on a synthetic dataset containing various metrics related to customer purchasing habits and engagement. Your goal will be to:
* Process and prepare the raw customer data.
* Use **Principal Component Analysis (PCA)** to understand the main drivers of customer variation.
* Employ **t-Distributed Stochastic Neighbor Embedding (t-SNE)** to uncover hidden clusters of similar customers.
* (Optional Challenge) Explore **Uniform Manifold Approximation and Projection (UMAP)** for an alternative perspective.
* Interpret these visualizations to describe potential customer segments and suggest business implications.

Let's begin by setting up our environment and loading our customer data!


# 1. Data Acquisition and Initial Exploration

For this tutorial, we will first work with a synthetic dataset named ecommerce_customer_data.csv. This file contains anonymized data representing various aspects of customer engagement and purchasing behavior over a period.

First, let's ensure we have our necessary libraries installed and then load the dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm.autonotebook import tqdm

In [ ]:
try:
    data = pd.read_csv("ecommerce_customer_data.csv")
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Creating synthetic dataset...")
    np.random.seed(42)
    num_customers = 500

    data = pd.DataFrame({
        'CustomerID': np.arange(1, num_customers + 1),
        'Age': np.random.randint(18, 70, num_customers),
        'Gender': np.random.choice(['Male', 'Female'], num_customers),
        'Average_Order_Value': np.random.normal(50, 20, num_customers).round(2).clip(min=5),
        'Number_of_Purchases': np.random.randint(1, 30, num_customers),
        'Days_Since_Last_Purchase': np.random.randint(1, 180, num_customers),
        'Product_Category_Preference': np.random.choice(['Electronics', 'Apparel', 'Books', 'Home Goods', 'Beauty'], num_customers),
        'Customer_Lifetime_Value': np.random.normal(200, 100, num_customers).round(2).clip(min=10)
    })

    data.to_csv("ecommerce_customer_data.csv", index=False)

In [ ]:
print("\nDataset Head:")
print(data.head())
print("\nDataset Info:")
data.info()
print("\nDataset Description:")
print(data.describe())

# 2. Feature Engineering and Preprocessing

Before we can apply dimensionality reduction techniques, we need to convert all our features into a numerical format and scale them appropriately. This is crucial because algorithms like PCA and t-SNE are sensitive to the magnitude of the features.

Here's our plan:
* **Drop CustomerID:** It's an identifier and doesn't contain behavioral information.
* **One-Hot Encode Categorical Features:** Convert Gender and Product_Category_Preference into numerical representations.
* **Standardize Numerical Features:** Scale all numerical features to have a mean of 0 and a standard deviation of 1.

In [ ]:
features_df = data.drop('CustomerID', axis=1)
features_df = pd.get_dummies(features_df, columns=['Gender', 'Product_Category_Preference'], drop_first=True)

numerical_cols = ['Age', 'Average_Order_Value', 'Number_of_Purchases',
                  'Days_Since_Last_Purchase', 'Customer_Lifetime_Value']

scaler = StandardScaler()
features_df[numerical_cols] = scaler.fit_transform(features_df[numerical_cols])

customer_labels = data['Product_Category_Preference']

Now our data features_df is ready for dimensionality reduction!

# 3. Dimensionality Reduction: Principal Component Analysis (PCA)

PCA is a linear dimensionality reduction technique that transforms the data into a new coordinate system where the greatest variance by any projection of the data comes to lie on the first coordinate (called the first principal component), the second greatest variance on the second coordinate, and so on. It helps us capture the most important information (variance) in fewer dimensions.

First, let's look at how much variance each principal component explains.

In [ ]:
pca = PCA()
pca.fit(features_df)
pca_data = pca.transform(features_df)
per_var = np.round(pca.explained_variance_ratio_ * 100, decimals=1)
labels_all = ['PC' + str(x) for x in range(1, len(per_var) + 1)]
per_var_display = per_var[:10]
labels_display = labels_all[:10]
with plt.style.context('dark_background'):
    plt.figure(figsize=(15, 7))
    plt.xlabel("Number of Principal Components")
    plt.ylabel("Percentage of variance explained")
    plt.bar(range(1, len(per_var_display) + 1), per_var_display, tick_label=labels_display, color="cyan", alpha=0.7)
    plt.plot(range(1, len(per_var_display) + 1), np.cumsum(per_var_display), color="red", marker='o', linestyle='--')
    plt.scatter(range(1, len(per_var_display) + 1), np.cumsum(per_var_display), color="yellow", s=50)
    plt.title("Explained Variance by Principal Components")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


**Observation:** The first few principal components capture a significant portion of the variance in our customer dataset. The cumulative variance curve shows how many components are needed to explain a certain amount of the total variation.

Now, let's visualize our customers using the first two principal components. We'll color the points by their Product_Category_Preference (which we saved earlier) to see if PCA naturally separates customers based on this known characteristic.

In [ ]:
pca_df = pd.DataFrame(data=pca_data[:, 0:2], columns=['PC1', 'PC2'])
pca_df['Product_Category_Preference'] = customer_labels.values
fig = px.scatter(pca_df, x='PC1', y='PC2', color='Product_Category_Preference',
                 title="Customer Segmentation via PCA (Colored by Product Preference)",
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
                 hover_data=['Product_Category_Preference'], # Add hover info
                 height=600, width=900,
                 color_discrete_sequence=px.colors.qualitative.Bold) # Use a nice color sequence
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)', # Transparent background
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)
fig.show(renderer="colab")

In [ ]:
pca_df = pd.DataFrame(pca_data[:, :2], columns=['PC1','PC2'])
pca_df['Category'] = customer_labels
fig = px.scatter(pca_df, x='PC1', y='PC2', color='Category')
fig.show()

Your turn to interpret!

**Observations from PCA Plot:**
* Do you see any clear separation based on Product_Category_Preference?
* Are there any dense clusters, even if they contain mixed preferences?
* What does the spread of points suggest about customer behavior?

PCA provides a good overall view, but it's a linear method. Sometimes, complex, non-linear relationships between data points are better captured by other techniques.

# 4. Dimensionality Reduction: t-Distributed Stochastic Neighbor Embedding (t-SNE)

t-SNE is a non-linear dimensionality reduction algorithm particularly well-suited for visualizing high-dimensional datasets. It aims to place data points in a low-dimensional space such that points that are close together in the high-dimensional space remain close together in the low-dimensional map, and points that are far apart remain far apart. t-SNE is excellent at revealing local structures and clusters.

A key parameter in t-SNE is perplexity. Perplexity relates to the number of nearest neighbors that are considered. It can be thought of as a continuous measure of the number of effective nearest neighbors. A good perplexity value often lies between 5 and 50. Different perplexity values can reveal different aspects of the data structure. n_iter defines the number of iterations for the optimization.

Let's apply t-SNE to our features_df and visualize the results.

In [ ]:
random_state = 42
n_components = 2
perplexity = 30
n_iter = 1000
print(f"Applying t-SNE with perplexity={perplexity}, n_iter={n_iter}...")
model_tsne = TSNE(n_components=n_components, random_state=random_state,
                  perplexity=perplexity, n_iter=n_iter, n_jobs=-1, verbose=1)
tsne_data = model_tsne.fit_transform(features_df) # Fix: Add this line to compute tsne_data
print("t-SNE completed.")
tsne_df = pd.DataFrame(data=tsne_data, columns=['TSNE1', 'TSNE2'])
tsne_df['Product_Category_Preference'] = customer_labels.values
fig = px.scatter(tsne_df, x='TSNE1', y='TSNE2', color='Product_Category_Preference',
                 title=f"Customer Segmentation via t-SNE (Perplexity={perplexity})",
                 labels={'TSNE1': 't-SNE Component 1', 'TSNE2': 't-SNE Component 2'},
                 hover_data=['Product_Category_Preference'],
                 height=600, width=900,
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)
fig.show(renderer="colab")

In [ ]:
model_tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
tsne_data = model_tsne.fit_transform(features_df)
tsne_df = pd.DataFrame(tsne_data, columns=['TSNE1','TSNE2'])
tsne_df['Category'] = customer_labels
fig = px.scatter(tsne_df, x='TSNE1', y='TSNE2', color='Category')
fig.show()

**Observations from t-SNE Plot:**
* How does this plot compare to the PCA plot? Is the separation of clusters more distinct?
* Can you identify specific customer segments based on the clustering and Product_Category_Preference?
* Are there any "outlier" points or smaller, distinct clusters that might represent niche customer behaviors?

**Experimentation Challenge:**
Try changing the perplexity parameter (e.g., to 5, 15, 50, or 100) and re-run the t-SNE code. How does this affect the clusters and overall structure of the plot? Which perplexity value seems to reveal the most interpretable customer segments?

**(Optional) 5. Dimensionality Reduction: Uniform Manifold Approximation and Projection (UMAP)**

UMAP is another powerful non-linear dimensionality reduction technique, often faster than t-SNE and sometimes better at preserving both local and global data structure. It's becoming increasingly popular for visualizing complex datasets.

To use UMAP, you might need to install it first: !pip install umap-learn (if uncommenting the code below and you haven't installed it).


In [ ]:
!pip install -q umap-learn
import umap
random_state = 42
n_components = 2
n_neighbors = 15
min_dist = 0.1
print(f"Applying UMAP with n_neighbors={n_neighbors}, min_dist={min_dist}...")
model_umap = umap.UMAP(n_components=n_components, random_state=random_state,
                     n_neighbors=n_neighbors, min_dist=min_dist, verbose=True)
umap_data = model_umap.fit_transform(features_df)
print("UMAP completed.")
umap_df = pd.DataFrame(data=umap_data, columns=['UMAP1', 'UMAP2'])
umap_df['Product_Category_Preference'] = customer_labels.values
fig = px.scatter(umap_df, x='UMAP1', y='UMAP2', color='Product_Category_Preference',
                 title=f"Customer Segmentation via UMAP (n_neighbors={n_neighbors}, min_dist={min_dist})",
                 labels={'UMAP1': 'UMAP Component 1', 'UMAP2': 'UMAP Component 2'},
                 hover_data=['Product_Category_Preference'],
                  height=600, width=900,
                  color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)
fig.show(renderer="colab")

**UMAP Observations:**
* If you ran the UMAP code, how do its clusters compare to t-SNE and PCA?
* Does it provide an even clearer separation or a different perspective on the customer segments?
* Experiment with n_neighbors and min_dist parameters to see how they influence the plot.

# 6. Conclusion and Business Implications

Congratulations! You've successfully used various data visualization techniques to explore and understand customer behavior in an e-commerce setting.

Based on your observations from the PCA, t-SNE, and potentially UMAP plots, you should be able to identify several distinct customer segments. For example:
* **High-Value Shoppers:** Customers with high Customer_Lifetime_Value and Average_Order_Value, potentially making frequent purchases. They might cluster together.
* **Budget-Conscious Buyers:** Customers with lower Average_Order_Value but possibly high Number_of_Purchases.
* **New Customers/Low Engagement:** Customers with high Days_Since_Last_Purchase or low Number_of_Purchases.
* **Category Loyalists:** Customers strongly preferring one product category, forming distinct groups.

**How would a business use these insights?**

Imagine presenting these plots to a marketing team. They could then:
* **Target High-Value Shoppers:** Offer exclusive early access to new products or personalized loyalty rewards.
* **Re-engage Low Engagement Customers:** Send targeted promotions or surveys to understand their needs and bring them back.
* **Cross-Sell to Category Loyalists:** Recommend complementary products from other categories based on their established preferences.
* **Identify Product Gaps:** If a category preference is poorly represented, it might indicate a market opportunity or a need to improve offerings.

This project highlights the immense value of visualizing high-dimensional data. Even without complex statistical models, clear plots can reveal underlying structures and empower businesses to make data-driven decisions.

Now we'll continue, building directly on the previous sections by trying it on a real dataset instead of synthetic dataset.

# Continuation: Applying Customer Segmentation to Real-World E-commerce Data
**Introduction: From Synthetic to Real-World Challenges**

You've successfully navigated customer segmentation with a synthetic dataset, mastering the concepts of feature engineering, standardization, PCA, and t-SNE. Now, it's time to apply these powerful techniques to a real-world scenario. Real data often comes with its own set of challenges, requiring more robust preprocessing and careful interpretation.

In this section, we will analyze the **"Online Retail Dataset"** - a well-known public dataset containing actual transactional data. This will allow us to:
* Experience data loading and cleaning for a more complex, real-world dataset.
* Derive meaningful features from raw transaction records.
* Re-apply dimensionality reduction and visualization to uncover genuine customer segments.
* Discuss the business implications based on real purchasing patterns.

Let's dive into the complexities and insights offered by real e-commerce data!

# 1. Real Data Acquisition and Initial Exploration: Online Retail Dataset

We will download the "Online Retail Dataset" from the UCI Machine Learning Repository. This dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retail company.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm.autonotebook import tqdm
try:
    # URL to the dataset (Excel file)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
    df_raw = pd.read_excel(url)
    print("Online Retail dataset downloaded and loaded successfully!")
except Exception as e:
    print(f"Error downloading or loading dataset: {e}")
    print("Please ensure you have 'openpyxl' installed: pip install openpyxl")
    # Fallback to a local file if download fails (e.g., if you've manually downloaded it)
    try:
        df_raw = pd.read_excel("Online Retail.xlsx")
        print("Loaded from local 'Online Retail.xlsx' file.")
    except FileNotFoundError:
        print("Local file 'Online Retail.xlsx' not found either. Please download it manually from:")
        print("https://archive.ics.uci.edu/ml/datasets/Online+Retail")
        print("And place it in the same directory as this notebook.")
        df_raw = pd.DataFrame() # Create an empty DataFrame to avoid errors later
if not df_raw.empty:
    print("\nDataset Head:")
    print(df_raw.head())
    print("\nDataset Info:")
    df_raw.info()
    print("\nDataset Description:")
    print(df_raw.describe())
else:
    print("\nCannot proceed without dataset. Please resolve the loading issue.")

This dataset is much larger and more complex! Key observations:

* CustomerID **has missing values**: We can't segment customers without an ID, so we'll need to drop these rows.
* Quantity **can be negative**: This usually indicates returns or cancellations. We should filter these out for purchase-based segmentation.
* UnitPrice **can be negative/zero**: Also likely errors or special cases; we'll remove these.
* InvoiceDate is a datetime object, which is good for time-based features.
Country is a categorical feature we might use for coloring.

# 2. Real Data Preprocessing and Feature Engineering (RFM Metrics)

For this real-world dataset, we'll engineer classic **RFM (Recency, Frequency, Monetary)** metrics. These are powerful features for customer segmentation:
* **Recency (R):** How recently did the customer make a purchase? (Days since last purchase)
* **Frequency (F):** How often do they purchase? (Total number of unique invoices)
* **Monetary (M):** How much money do they spend? (Total spend)

Now we do preprocessing and feature engineering

1. **Clean Data:**
   * Remove rows with missing CustomerID.
   * Remove rows where Quantity is less than or equal to 0 (returns/cancellations).
   * Remove rows where UnitPrice is less than or equal to 0.
2. **Calculate Total Price:** Quantity * UnitPrice.
3. **Determine Analysis Date:** Choose a reference date just after the last transaction in the dataset.
4. **Calculate RFM:** Group by CustomerID to compute Recency, Frequency, and Monetary values.
5. **Standardize Features:** Apply StandardScaler to RFM values.

In [ ]:
df = df_raw.copy()
df.dropna(subset=['CustomerID'], inplace=True)
df['CustomerID'] = df['CustomerID'].astype(int)
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
print(f"Cleaned data shape: {df.shape}")
print(f"Number of unique customers: {df['CustomerID'].nunique()}")
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
max_invoice_date = df['InvoiceDate'].max()
analysis_date = max_invoice_date + pd.Timedelta(days=1)
print(f"Analysis Reference Date: {analysis_date}")
rfm_df = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda date: (analysis_date - date.max()).days),
    Frequency=('InvoiceNo', 'nunique'), # Count unique invoices for frequency
    Monetary=('TotalPrice', 'sum')
).reset_index()
print("\nRFM Features Head:")
print(rfm_df.head())
print("\nRFM Features Description:")
print(rfm_df.describe())
customer_country = df.drop_duplicates(subset=['CustomerID']).set_index('CustomerID')['Country']
rfm_df['Country'] = rfm_df['CustomerID'].map(customer_country)
rfm_features = rfm_df.drop(['CustomerID', 'Country'], axis=1)
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)
rfm_scaled_df = pd.DataFrame(rfm_scaled, columns=rfm_features.columns, index=rfm_df.index)
print("\nScaled RFM Features Head:")
print(rfm_scaled_df.head())

Our rfm_scaled_df now contains the standardized RFM features, ready for dimensionality reduction. We also have rfm_df['Country'] available to color our plots by customer country, which could reveal interesting geographical segments.

# 3. Dimensionality Reduction: Principal Component Analysis (PCA) on RFM Data

Let's re-apply PCA to our RFM features. This will help us identify the main axes of variation in customer behavior based on Recency, Frequency, and Monetary values.

In [ ]:
pca_rfm = PCA()
pca_rfm.fit(rfm_scaled_df)
pca_rfm_data = pca_rfm.transform(rfm_scaled_df)
per_var_rfm = np.round(pca_rfm.explained_variance_ratio_ * 100, decimals=1)
labels_all_rfm = ['PC' + str(x) for x in range(1, len(per_var_rfm) + 1)]
per_var_rfm_display = per_var_rfm[:3]
labels_rfm_display = labels_all_rfm[:3]
with plt.style.context('dark_background'):
    plt.figure(figsize=(10, 6))
    plt.xlabel("Number of Principal Components")
    plt.ylabel("Percentage of variance explained")
    plt.bar(range(1, len(per_var_rfm_display) + 1), per_var_rfm_display,
            tick_label=labels_rfm_display, color="lightgreen", alpha=0.7)
    plt.plot(range(1, len(per_var_rfm_display) + 1), np.cumsum(per_var_rfm_display),
             color="red", marker='o', linestyle='--')
    plt.scatter(range(1, len(per_var_rfm_display) + 1), np.cumsum(per_var_rfm_display),
                color="yellow", s=50)
    plt.title("Explained Variance by Principal Components (RFM Data)")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

**Observation:** With only three features (R, F, M), PCA is straightforward. The first PC typically explains a large portion, but all three components are often needed to capture most of the variance.

Now, let's visualize the customers in the 2D PCA space, coloring them by Country to see if geographic location plays a role in customer behavior patterns. We'll focus on the top 10 countries by customer count to keep the legend manageable, and group others as 'Other'.

In [ ]:
pca_rfm_df = pd.DataFrame(data=pca_rfm_data[:, 0:2], columns=['PC1', 'PC2'])
pca_rfm_df['CustomerID'] = rfm_df['CustomerID'] # Keep CustomerID for merging
customer_country_data = rfm_df[['CustomerID', 'Country']]
pca_rfm_df = pd.merge(pca_rfm_df, customer_country_data, on='CustomerID', how='left')
top_countries = pca_rfm_df['Country'].value_counts().nlargest(10).index
pca_rfm_df['Country_Grouped'] = pca_rfm_df['Country'].apply(lambda x: x if x in top_countries else 'Other')
fig = px.scatter(pca_rfm_df, x='PC1', y='PC2', color='Country_Grouped',
                 title="Customer Segmentation via PCA (RFM - Colored by Country)",
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
                 hover_data=['CustomerID', 'Country_Grouped', 'PC1', 'PC2'],
                 height=700, width=1000,
                 color_discrete_sequence=px.colors.qualitative.Alphabet) # Use a good color sequence
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)
fig.show(renderer="colab")

The next natural step is to apply t-SNE and then UMAP to the real RFM data.

This will allow us to compare how these non-linear methods perform in revealing customer segments compared to PCA, especially with real-world complexities.

Let's continue with t-SNE:

# 4. Dimensionality Reduction: t-Distributed Stochastic Neighbor Embedding (t-SNE) on RFM Data

As we saw with the synthetic data, t-SNE excels at uncovering non-linear relationships and local clusters within the data. With our real RFM features, t-SNE should provide a more nuanced view of customer segments compared to the linear PCA.

We'll use the same perplexity and n_iter parameters as a starting point, but remember that experimenting with these values is key to finding the most insightful visualization for your specific dataset.

In [ ]:
random_state = 42
n_components = 2
perplexity = 30
n_iter = 1000
print(f"Applying t-SNE to RFM data with perplexity={perplexity}, n_iter={n_iter}...")
model_tsne_rfm = TSNE(n_components=n_components, random_state=random_state,
                      perplexity=perplexity, n_iter=n_iter, n_jobs=-1, verbose=1)
tsne_rfm_data = model_tsne_rfm.fit_transform(rfm_scaled_df)
print("t-SNE on RFM data completed.")
tsne_rfm_df = pd.DataFrame(data=tsne_rfm_data, columns=['TSNE1', 'TSNE2'])
tsne_rfm_df['CustomerID'] = rfm_df['CustomerID']
tsne_rfm_df = pd.merge(tsne_rfm_df, customer_country_data, on='CustomerID', how='left')
tsne_rfm_df['Country_Grouped'] = tsne_rfm_df['Country'].apply(lambda x: x if x in top_countries else 'Other')
fig = px.scatter(tsne_rfm_df, x='TSNE1', y='TSNE2', color='Country_Grouped',
                 title=f"Customer Segmentation via t-SNE (RFM - Perplexity={perplexity})",
                 labels={'TSNE1': 't-SNE Component 1', 'TSNE2': 't-SNE Component 2'},
                 hover_data=['CustomerID', 'Country_Grouped', 'TSNE1', 'TSNE2'],
                 height=700, width=1000,
                 color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)
fig.show(renderer="colab")

**Observations from t-SNE Plot on RFM Data:**
* How do the clusters here compare to the PCA plot? Is the separation generally better defined?
* Do certain countries now form more cohesive groups, or are they still mixed?
* Can you visually identify distinct customer behavior segments (e.g., a tight cluster of high-frequency buyers vs. a dispersed group of infrequent purchasers)?

**Experimentation Challenge (Important!):**

Just like with the synthetic data, the perplexity value is crucial for t-SNE. Re-run the t-SNE code cell with different perplexity values (e.g., 5, 15, 50, 100, 200). Observe how the clustering changes. Which perplexity value do you think gives the most meaningful and stable representation of customer segments in this real dataset? Why?

# 5. Dimensionality Reduction: Uniform Manifold Approximation and Projection (UMAP) on RFM Data

Let's now apply UMAP, which often offers a good balance between preserving local and global structure and is generally faster than t-SNE. We'll continue to color by Country_Grouped.

In [ ]:
import umap
import pandas as pd
import plotly.express as px
random_state = 42
n_components = 2
n_neighbors = 15
min_dist = 0.1
print(f"Applying UMAP to RFM data with n_neighbors={n_neighbors}, min_dist={min_dist}...")
model_umap_rfm = umap.UMAP(
    n_components=n_components,
    random_state=random_state,
    n_neighbors=n_neighbors,
    min_dist=min_dist,
    verbose=True
)

umap_rfm_data = model_umap_rfm.fit_transform(rfm_scaled_df)
print("UMAP on RFM data completed.")
umap_rfm_df = pd.DataFrame(umap_rfm_data, columns=['UMAP1', 'UMAP2'])
umap_rfm_df['CustomerID'] = rfm_df['CustomerID']
umap_rfm_df = umap_rfm_df.merge(customer_country_data, on='CustomerID', how='left')
umap_rfm_df['Country_Grouped'] = umap_rfm_df['Country'].apply(
    lambda x: x if x in top_countries else 'Other'
)
fig = px.scatter(
    umap_rfm_df,
    x='UMAP1',
    y='UMAP2',
    color='Country_Grouped',
    title=f"Customer Segmentation via UMAP (RFM — n_neighbors={n_neighbors}, min_dist={min_dist})",
    labels={'UMAP1': 'UMAP Component 1', 'UMAP2': 'UMAP Component 2'},
    hover_data=['CustomerID', 'Country_Grouped', 'UMAP1', 'UMAP2'],
    height=700,
    width=1000,
    color_discrete_sequence=px.colors.qualitative.Alphabet
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font_color='white'
)

fig.show()

**UMAP Observations on RFM Data:**
* In case you have uncommented and run UMAP, how does UMAP's representation of the clusters compare to both PCA and t-SNE? Does it show a clearer global structure or sharper local clusters?
* Are there any "bridges" or connections between clusters that UMAP highlights better than t-SNE?
* Consider how different n_neighbors (e.g., 5, 50, 100) and min_dist (e.g., 0.0, 0.5) values might alter the UMAP embedding.